In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 44.2 MB/s eta 0:00:00


In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer
import json

In [ ]:
# !huggingface-cli login

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
# model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Data Pre-Processing

In [ ]:
DEFAULT_SYSTEM_PROMPT ="""Given the following SQL tables, your job is to write a queries given a user’s request. If you think you cannot get the correct SQL, answer with 'null'.

CREATE TABLE admissions ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL UNIQUE,admittime TIMESTAMP(0) NOT NULL, dischtime TIMESTAMP(0), admission_type VARCHAR(50) NOT NULL, admission_location VARCHAR(50) NOT NULL, discharge_location VARCHAR(50), insurance VARCHAR(255) NOT NULL, language VARCHAR(10), marital_status VARCHAR(50), age INT NOT NULL, FOREIGN KEY(subject_id) REFERENCES patients(subject_id));
CREATE TABLE chartevents ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, stay_id INT NOT NULL,itemid INT NOT NULL, charttime TIMESTAMP(0) NOT NULL, valuenum DOUBLE PRECISION, valueuom VARCHAR(50), FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id), FOREIGN KEY(stay_id) REFERENCES icustays(stay_id), FOREIGN KEY(itemid) REFERENCES d_items(itemid) );
CREATE TABLE cost ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, event_type VARCHAR(20) NOT NULL, event_id INT NOT NULL, chargetime TIMESTAMP(0) NOT NULL, cost DOUBLE PRECISION NOT NULL, FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id), FOREIGN KEY(event_id) REFERENCES diagnoses_icd(row_id), FOREIGN KEY(event_id) REFERENCES procedures_icd(row_id), FOREIGN KEY(event_id) REFERENCES labevents(row_id), FOREIGN KEY(event_id) REFERENCES prescriptions(row_id));
CREATE TABLE d_icd_diagnoses ( row_id INT NOT NULL PRIMARY KEY, icd_code VARCHAR(10) NOT NULL UNIQUE, long_title VARCHAR(255) NOT NULL);
CREATE TABLE d_icd_procedures ( row_id INT NOT NULL PRIMARY KEY, icd_code VARCHAR(10) NOT NULL UNIQUE, long_title VARCHAR(255) NOT NULL);
CREATE TABLE d_items ( row_id INT NOT NULL PRIMARY KEY, itemid INT NOT NULL UNIQUE, label VARCHAR(200) NOT NULL, abbreviation VARCHAR(200) NOT NULL, linksto VARCHAR(50) NOT NULL);
CREATE TABLE d_labitems (row_id INT NOT NULL PRIMARY KEY, itemid INT NOT NULL UNIQUE, label VARCHAR(200));
CREATE TABLE diagnoses_icd ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, icd_code VARCHAR(10) NOT NULL, charttime TIMESTAMP(0) NOT NULL, FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id), FOREIGN KEY(icd_code) REFERENCES d_icd_diagnoses(icd_code));
CREATE TABLE icustays ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, stay_id INT NOT NULL UNIQUE, first_careunit VARCHAR(20) NOT NULL, last_careunit VARCHAR(20) NOT NULL, intime TIMESTAMP(0) NOT NULL, outtime TIMESTAMP(0), FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id) );
CREATE TABLE inputevents ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, stay_id INT NOT NULL, starttime TIMESTAMP(0) NOT NULL, itemid INT NOT NULL, amount DOUBLE PRECISION, FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id), FOREIGN KEY(stay_id) REFERENCES icustays(stay_id), FOREIGN KEY(itemid) REFERENCES d_items(itemid));
CREATE TABLE labevents ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, mitemid INT NOT NULL, charttime TIMESTAMP(0), valuenum DOUBLE PRECISION, valueuom VARCHAR(20), FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id), FOREIGN KEY(itemid) REFERENCES d_labitems(itemid));
CREATE TABLE microbiologyevents ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, charttime TIMESTAMP(0) NOT NULL, spec_type_desc VARCHAR(100), test_name VARCHAR(100), org_name VARCHAR(100), FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id));
CREATE TABLE outputevents ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, stay_id INT NOT NULL, charttime TIMESTAMP(0) NOT NULL, itemid INT NOT NULL, value DOUBLE PRECISION, FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id), FOREIGN KEY(stay_id) REFERENCES icustays(stay_id), FOREIGN KEY(itemid) REFERENCES d_items(itemid) );
CREATE TABLE patients ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL UNIQUE, gender VARCHAR(5) NOT NULL, dob TIMESTAMP(0) NOT NULL, dod TIMESTAMP(0));
CREATE TABLE prescriptions ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, starttime TIMESTAMP(0) NOT NULL, stoptime TIMESTAMP(0), drug VARCHAR(255) NOT NULL, dose_val_rx VARCHAR(100) NOT NULL, dose_unit_rx VARCHAR(50) NOT NULL, route VARCHAR(50) NOT NULL, FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id));
CREATE TABLE procedures_icd ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, icd_code VARCHAR(10) NOT NULL, charttime TIMESTAMP(0) NOT NULL, FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id), FOREIGN KEY(icd_code) REFERENCES d_icd_procedures(icd_code));
CREATE TABLE transfers ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, transfer_id INT NOT NULL, eventtype VARCHAR(20) NOT NULL, careunit VARCHAR(20), intime TIMESTAMP(0) NOT NULL, outtime TIMESTAMP(0), FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id));
""".strip()

# List of all tables
tables = [
    "admissions", "chartevents", "cost", "d_icd_diagnoses", "d_icd_procedures",
    "d_items", "d_labitems", "diagnoses_icd", "icustays", "inputevents",
    "labevents", "microbiologyevents", "outputevents", "patients",
    "prescriptions", "procedures_icd", "transfers"
]


In [ ]:
def extract_table_definition(table_name, prompt):
    start = prompt.find(f"CREATE TABLE {table_name}")
    if start == -1:
        return None
    end = prompt.find("CREATE TABLE", start + 1)
    if end == -1:
        end = len(prompt)
    return prompt[start:end].strip()

def extract_relevant_foreign_keys(table_list, foreign_keys):
    relevant_keys = []
    for key in foreign_keys:
        # Split the foreign key on '=' and then further split on '.' to isolate table names
        tables_in_key = set([part.strip().split('.')[0] for part in key.replace(" ", "").split('=')])
        # Check if all tables in the foreign key are in the provided table list
        if all(table in table_list for table in tables_in_key):
            relevant_keys.append(key)
    return relevant_keys

In [ ]:
def construct_custom_system_prompt(sql_query, original_prompt, table_list):
    included_tables = []
    foreign_keys_tables = []
    for table in table_list:
        if table in sql_query:
            table_def = extract_table_definition(table, original_prompt)
            foreign_keys_tables.append(table)
            if table_def:
                included_tables.append(table_def)

    new_prompt = "Given the following SQL tables, your job is to write a sql query for a given user’s request. If you think you cannot get the correct SQL, answer with 'null'.\n\n"
    new_prompt += "\n".join(included_tables)
    new_prompt += "\n\n #diction \n SQL \n"
    return new_prompt.strip()


In [ ]:
questions = json.load(open('/content/drive/MyDrive/Colab_Notebooks/CSCI 5922/final project/data_text2sql/mimic_iv/train/data.json'))['data']
sql_queries = json.load(open('/content/drive/MyDrive/Colab_Notebooks/CSCI 5922/final project/data_text2sql/mimic_iv/train/label.json'))
aug_data = json.load(open('/content/drive/MyDrive/Colab_Notebooks/CSCI 5922/final project/data_text2sql/augmented_data.json'))
# questions = json.load(open('/content/drive/MyDrive/Colab_Notebooks/CSCI 5922/final project/data_text2sql/mimic_iv/test/data.json'))['data'] #test data
# sql_queries = json.load(open('/content/drive/MyDrive/Colab_Notebooks/CSCI 5922/final project/data_text2sql/mimic_iv/test/label.json')) # test data

In [ ]:
from typing import List
def merge_aug_datasets(questions: List[dict], sql_queries: List[dict], tables: List[str], system_prompt: str = DEFAULT_SYSTEM_PROMPT):
    merged_dataset = []
    for question_item in questions:
        item_id = question_item['og_id']
        new_item_id = question_item['id']
        if item_id in sql_queries:
            merged_item = {
                'id': new_item_id,
                "system_prompt": construct_custom_system_prompt(sql_queries[item_id], system_prompt, tables),
                'question': question_item['question'],
                'sql_query': sql_queries[item_id],
            }
            merged_dataset.append(merged_item)
    return merged_dataset

def merge_datasets(questions: List[dict], sql_queries: List[dict], tables: List[str], system_prompt: str = DEFAULT_SYSTEM_PROMPT):
    merged_dataset = []
    for question_item in questions:
        item_id = question_item['id']
        if item_id in sql_queries:
            merged_item = {
                'id': item_id,
                "system_prompt": construct_custom_system_prompt(sql_queries[item_id], system_prompt, tables),
                'question': question_item['question'],
                'sql_query': sql_queries[item_id]
            }
            merged_dataset.append(merged_item)
    return merged_dataset

In [ ]:
merged_dataset = merge_datasets(questions, sql_queries, tables)
merged_aug_dataset = merge_aug_datasets(aug_data, sql_queries, tables)
combined_dataset = merged_dataset + merged_aug_dataset
# combined_dataset = merged_dataset

In [ ]:
print(combined_dataset[10]['system_prompt'])
print(combined_dataset[10]['question'])
print(combined_dataset[10]['sql_query'])

Given the following SQL tables, your job is to write a sql query for a given user’s request. If you think you cannot get the correct SQL, answer with 'null'.

CREATE TABLE admissions ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL UNIQUE,admittime TIMESTAMP(0) NOT NULL, dischtime TIMESTAMP(0), admission_type VARCHAR(50) NOT NULL, admission_location VARCHAR(50) NOT NULL, discharge_location VARCHAR(50), insurance VARCHAR(255) NOT NULL, language VARCHAR(10), marital_status VARCHAR(50), age INT NOT NULL, FOREIGN KEY(subject_id) REFERENCES patients(subject_id));
CREATE TABLE microbiologyevents ( row_id INT NOT NULL PRIMARY KEY, subject_id INT NOT NULL, hadm_id INT NOT NULL, charttime TIMESTAMP(0) NOT NULL, spec_type_desc VARCHAR(100), test_name VARCHAR(100), org_name VARCHAR(100), FOREIGN KEY(hadm_id) REFERENCES admissions(hadm_id));

 #diction 
 SQL
What was the name of the organism that was found in the last microbiology test on patient 10007795's abscess?

In [ ]:
import datasets
merged_training_dataset = datasets.Dataset.from_list(combined_dataset[:20])
merged_training_dataset = merged_training_dataset.shuffle(seed=64)

In [ ]:
# merged_training_dataset

In [ ]:
# split_datasets = merged_training_dataset.train_test_split(test_size=0.2)  # Adjust test_size as needed

# # Access the training and test datasets
# train_dataset = split_datasets['train']
# test_dataset = split_datasets['test']

# # Example: Show the size of each split
# print(f"Training dataset size: {len(train_dataset)}")
# print(f"Test dataset size: {len(test_dataset)}")

# Training

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/Colab_Notebooks/CSCI 5922/final project/llama2_all_data_2"

In [ ]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 16

# Batch size per GPU for evaluation
per_device_eval_batch_size = 16

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-5

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 500

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 1024

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
from peft import prepare_model_for_kbit_training

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training



In [ ]:
print("\nEOS token: ", tokenizer.eos_token)
print("EOS token id:", tokenizer.eos_token_id)
print("\nPad token: ", tokenizer.pad_token)
print("Pad token id: ", tokenizer.pad_token_id)

In [ ]:
if '|<pad>|' not in tokenizer.get_vocab():

  #Add pad token
  tokenizer.add_tokens(['|<pad>|'])

#set the pad token
tokenizer.pad_token = '|<pad>|'

#resize token embeddings
model.resize_token_embeddings(len(tokenizer))

#update pad token id in model and its config
model.pad_token_id = tokenizer.pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id


#check that equality
assert model.pad_token_id == tokenizer.pad_token_id, "The model's pad token ID does not match the tokenizer's pad token"

print("tokenizer pad token ID: ", tokenizer.pad_token_id)
print("Model pad token ID: ", model.pad_token_id)
print("Model config pad token ID: ", model.config.pad_token_id)

print(model.config)

In [ ]:
 sample_string = ['[INST]']

encoded_sample = tokenizer(sample_string, truncation=True, padding= True, max_length=1024, return_attention_mask=True)

token_count = len(encoded_sample)

BOS_token_id = tokenizer.bos_token_id
EOS_token_id = tokenizer.eos_token_id

BOS_token = tokenizer.decode([BOS_token_id])
EOS_token = tokenizer.decode([EOS_token_id])


print(f"Beginning of the sequence: {sample_string[0]} (BOS token: {BOS_token}), id: {BOS_token_id}")
print(f"End of the sequence: {sample_string[-1]} (EOS token: {EOS_token}, id: {EOS_token_id})")

print(f"The number of tokens in the string is: {token_count}")
print(f"the ids are: {encoded_sample}")


decoded_sample = tokenizer.decode(encoded_sample['input_ids'][0], skip_special_tokens=False)

print(f"the decoded string is {decoded_sample}")

In [ ]:
class TextDataset(Dataset):
  def __init__(self, encodings, response_lengths):
    self.encodings = encodings
    self.response_lengths = response_lengths

  def __getitem__(self, idx):
    item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}

    #Set labels to the same as input_ids
    item['labels'] = item['input_ids'].clone()


    #Shift labels to the left and replace the last position with EOS token
    item['labels'][:-1] = item['input_ids'][1:]
    item['labels'][-1] = 2 #Replace last position with EOS token ID


    #Create a loss mask
    response_start_position = item['input_ids'].shape[0] - self.response_lengths[idx]
    item['loss_mask'] = torch.zeros_like(item["input_ids"])
    item['loss_mask'][response_start_position:] = 1

    #create a new tensor for the shifted loss mask
    shifted_loss_mask = torch.cat([item['loss_mask'][1:], torch.tensor([1])])
    item['loss_mask'] = shifted_loss_mask

    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

In [ ]:
class TextDataset_Right_Padding(Dataset):
  def __init__(self, encodings, response_lengths):
    self.encodings = encodings
    self.response_lengths = response_lengths

  def __getitem__(self, idx):
    item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}

    # Set labels to the same as input_ids
    item['labels'] = item['input_ids'].clone()
    # Find the index of the first padding token
    padding_idx = 32000  # Assuming 0 is the padding token index
    first_pad_index = (item['input_ids'] == padding_idx).nonzero(as_tuple=True)[0][0]

    # Calculate the actual end of the sequence before padding
    actual_end = first_pad_index

    # Shift labels to the left by one position up to the actual end of the sequence
    item['labels'][:actual_end-1] = item['input_ids'][1:actual_end]
    item['labels'][actual_end-1] = 2  # Place EOS token at the end of the actual sequence


    # Create a loss mask that is 1 for the actual response, excluding padding
    item['loss_mask'] = torch.zeros_like(item["input_ids"])
    response_start_position = first_pad_index - self.response_lengths[idx]
    item['loss_mask'][response_start_position:first_pad_index] = 1

    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

In [ ]:
def prepare_dataset(dataset, tokenizer, max_length=1024):
    # Define the roles and markers
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

    # Apply transformation to each item in the dataset
    formatted_dataset = dataset.map(
        lambda x: {
            "input_text": "".join([
                f"{B_INST} {B_SYS} {x['system_prompt'].strip()} {E_SYS} ",
                f"{x['question'].strip()} {E_INST} ",
                f"{x['sql_query'].strip()}"  # appending the EOS token in text data...
            ]),
            "response_text": "".join([
                f"{x['sql_query'].strip()}"  # appending the EOS token in text data...
            ])
        }
    )
    #tokenize the dataset
    encodings = tokenizer([dialogue['input_text'] for dialogue in formatted_dataset], truncation=True, return_tensors='pt', max_length=max_length, padding=True)

    response_length = [len(tokenizer.encode(dialogue['response_text'], truncation = True, max_length=max_length)) for dialogue in formatted_dataset]

    text_dataset = TextDataset_Right_Padding(encodings, response_length)
    return text_dataset


In [ ]:
train_data = prepare_dataset(merged_training_dataset, tokenizer)
# test_data = prepare_dataset(test_dataset, tokenizer)

In [ ]:
train_data

In [ ]:
sample_item = train_data[50]

print(f"Dimensions of input_ids: {sample_item['input_ids'].shape}")
print(f"Dimensions of attention_mask: {sample_item['attention_mask'].shape}")
print(f"Dimensions of loss_mask: {sample_item['loss_mask'].shape}")
print(f"Dimensions of labels: {sample_item['labels'].shape}")

num_tokens_to_print = 200

print("\nTokens at the start of the sample:")
print(sample_item['input_ids'][:num_tokens_to_print].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['input_ids'][:num_tokens_to_print].tolist()))

print("\nLabels at the start of the sample:")
print(sample_item['labels'][:num_tokens_to_print].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['labels'][:num_tokens_to_print].tolist()))

print("\nAttention Mask at the start of the sample:")
print(sample_item['attention_mask'][:num_tokens_to_print].tolist())

print("\nLoss Mask at the start of the sample:")
print(sample_item['loss_mask'][:num_tokens_to_print].tolist())



print("\nTokens at the end of the sample:")
print(sample_item['input_ids'][-num_tokens_to_print:].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['input_ids'][-num_tokens_to_print:].tolist()))

print("\nLabels at the end of the sample:")
print(sample_item['labels'][-num_tokens_to_print:].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['labels'][-num_tokens_to_print:].tolist()))

print("\nAttention Mask at the end of the sample:")
print(sample_item['attention_mask'][-num_tokens_to_print:].tolist())

print("\nLoss Mask at the end of the sample:")
print(sample_item['loss_mask'][-num_tokens_to_print:].tolist())

IndexError: index 50 is out of bounds for dimension 0 with size 20

In [ ]:
loss_mask_list = sample_item['loss_mask'].tolist()
first_non_zero_loss_id = loss_mask_list.index(1)
last_non_zero_loss_id = first_non_zero_loss_id
for i in range(first_non_zero_loss_id, len(loss_mask_list)):
  if loss_mask_list[i] == 1:
    last_non_zero_loss_id = i
  else:
    break

In [ ]:
print(first_non_zero_loss_id)
print(sample_item['input_ids'].tolist()[first_non_zero_loss_id-5:first_non_zero_loss_id+5])
print(tokenizer.convert_ids_to_tokens(sample_item['input_ids'].tolist()[first_non_zero_loss_id-5:first_non_zero_loss_id+5]))
print(sample_item['labels'].tolist()[first_non_zero_loss_id])
print(tokenizer.convert_ids_to_tokens(sample_item['labels'].tolist()[first_non_zero_loss_id]))

NameError: name 'first_non_zero_loss_id' is not defined

In [ ]:
print(last_non_zero_loss_id)
print(sample_item['input_ids'].tolist()[last_non_zero_loss_id])
print(tokenizer.convert_ids_to_tokens(sample_item['input_ids'].tolist()[last_non_zero_loss_id]))
print(sample_item['labels'].tolist()[last_non_zero_loss_id])
print(tokenizer.convert_ids_to_tokens(sample_item['labels'].tolist()[last_non_zero_loss_id]))

print(tokenizer.convert_ids_to_tokens(sample_item['input_ids'].tolist()[last_non_zero_loss_id - 5: last_non_zero_loss_id+5]))
print(sample_item['labels'].tolist()[last_non_zero_loss_id])
print(tokenizer.convert_ids_to_tokens(sample_item['labels'].tolist()[last_non_zero_loss_id-5 : last_non_zero_loss_id+5]))

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
# print(model.state_dict().keys())
# model.state_dict().keys()

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",)

model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 33554432 || all params: 3533975552 || trainable%: 0.9494811581537506


In [ ]:
import json
from transformers import TrainerCallback

class LossLoggingCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.losses = []
        self.path = os.path.join(OUTPUT_DIR, 'losses.json')

    def on_log(self, args, state, control, logs=None, **kwargs):
        if 'loss' in logs:
            self.losses.append({'step': state.global_step, 'loss': logs['loss']})

    def on_train_end(self, args, state, control, **kwargs):
        with open(self.path , 'w') as f:
            json.dump(self.losses, f, indent=4)

In [ ]:
import torch.nn as nn
import transformers

class CustomTrainer(transformers.Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    #Define the number of tokens you want to display
    num_tokens = 25 #this displays info on the actual and predicted tyokens at the end of each....

    labels = inputs.pop('labels')
    loss_mask = inputs.pop("loss_mask")

    # Forward pass
    outputs = model(**inputs)

    logits = outputs.logits

    # Check for Nan in logits and labels
    if torch.isnan(logits).any():
      print("NaN detected in logits")
      print(logits)

    # Convert logits to probabilities using softmax function
    probs = nn.functional.softmax(logits, dim=-1)

    # Get the most probable tokens
    predicted_token_ids = torch.argmax(probs, dim=-1)

    # Compute the loss
    loss_fct = nn.CrossEntropyLoss(reduction='none')
    losses = loss_fct(logits.view(-1, self.model.config.vocab_size), labels.view(-1))

    #Reshaping the losses to have dimensions [ batch_size, seq_length]
    losses = losses.view(-1, inputs['input_ids'].size(1))

    #Apply the loss mask
    masked_loss = losses * loss_mask

    #Check for NaN in losses and zero in loss_mask.sum()
    if torch.isnan(losses).any():
      print("NaN detected in losses")
      # print(losses)

    if loss_mask.sum() == 0:
      print("Sum of loss_mask is zero")
      return (torch.tensor(0.0).to(loss_mask.device), outputs) if return_outputs else torch.tensor(0.0).to(loss_mask.device) # last section was autogenerated

    # Aggregate the masked losses
    loss = masked_loss.sum() / ( loss_mask.sum() + 1e-9) #normalizing by number of tokens

    #Print formatted tokens
    batch_size, seq_length = inputs['input_ids'].size()
    # print("-" * 120)
    # print(f"Token analysis for last {num_tokens} tokens:")

    # header_format = "{:<10}{:<20}{:<20}{:<20}{:<20}{:<30}{:<30}".format("Index", "Input Token",  ) #need

    for batch_idx in range(batch_size):
      input_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][batch_idx])
      predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_ids[batch_idx])
      true_tokens = tokenizer.convert_ids_to_tokens(labels[batch_idx])

      for i in range(-num_tokens, 0, 1):
        index = seq_length + i #correct index based on sequence length

    return (loss, outputs ) if return_outputs else loss


  def get_train_dataloader(self):
    train_dataset = self.train_dataset
    data_collator = self.data_collator

    dataloader_params = {
        "batch_size" : self.args.train_batch_size,
        "collate_fn" : data_collator,
        "num_workers" : self.args.dataloader_num_workers,
        "pin_memory" : self.args.dataloader_pin_memory,
    }

    if not isinstance(train_dataset, torch.utils.data.IterableDataset):
      dataloader_params['sampler'] = self._get_train_sampler()
      dataloader_params['drop_last'] = self.args.dataloader_drop_last

    return DataLoader(train_dataset, **dataloader_params)

In [ ]:
class CustomDataCollator:
    def __init__(self, tokenizer) -> None:
        self.tokenizer = tokenizer
    def __call__(self, batch):
        input_ids = torch.stack([x['input_ids'] for x in batch])
        attention_mask = torch.stack([x['attention_mask'] for x in batch])
        labels = torch.stack([x['labels'] for x in batch])
        loss_mask = torch.stack([x['loss_mask'] for x in batch])
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
            'loss_mask': loss_mask
        }
data_collator = CustomDataCollator(tokenizer)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)
trainer = CustomTrainer(
    model=model,
    data_collator=data_collator,
    args=training_arguments,
    train_dataset=train_data
)

trainer.add_callback(LossLoggingCallback())

In [ ]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 568.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 358.81 MiB is free. Process 83828 has 39.21 GiB memory in use. Of the allocated memory 38.13 GiB is allocated by PyTorch, and 599.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.save_model()

In [ ]:
# import re

# def extract_after_token(text, token='[/INST]'):
#     # Search for the token in the text
#     match = re.search(re.escape(token), text)

#     # If the token is found, return everything after it
#     if match:
#         return text[match.end():].strip()
#     else:
#         return "Token not found."

In [ ]:
# from transformers import TextStreamer
# # Define a stream *without* function calling capabilities
# def run_inference(system_prompt, query):
#     runtimeFlag = "cuda:0"


#     B_INST, E_INST = "[INST]", "[/INST]"
#     B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

#     prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{query.strip()} {E_INST}\n\n"

#     inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

#     # streamer = TextStreamer(tokenizer)

#     # # Despite returning the usual output, the streamer will also print the generated text to stdout.
#     # _ = model.generate(**inputs, streamer=streamer, max_new_tokens=500)
#     outputs = model.generate(**inputs, max_new_tokens=500)

#     # Decode and print the generated text
#     generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return extract_after_token(generated_text)

In [ ]:
# len(test_dataset)

In [ ]:
# data = []

# for num in range(len(test_dataset)):
#   entry = test_dataset[num]
#   system = entry['system_prompt']
#   query = entry['question']
#   ground_truth = entry['sql_query']
#   output = run_inference(system, query)

#   record = {
#     'id' : id,
#     'system_prompt': system,
#     'question' : query,
#     'ground_truth': ground_truth,
#     'out_put' : output
#   }
#   data.append(record)

# filename = os.path.join(OUTPUT_DIR, 'answers.json')  # Using os.path.join for path construction

# # Try to write to the file, with error handling:
# try:
#     with open(filename, 'w') as file:
#         json.dump(data, file, indent=4)
# except IOError as e:
#     print(f"An error occurred trying to write the file: {str(e)}")
# except Exception as e:
#     print(f"An unexpected error occurred: {str(e)}")

# Old method

### Trying out script to merge model weights starting here

In [ ]:
trainer.save_model()

In [ ]:
# base_model = AutoModelForCausalLM.from_pretrained(
#       model_name,
#       use_safetensors=True,
#       trust_remote_code=True,
#   ).to('cuda:0')

In [ ]:
# combined_model = PeftModel.from_pretrained(model, OUTPUT_DIR, torch_dtype=torch.float16)
# print(f"Running merge_and_unload")
# combined_model = combined_model.merge_and_unload()

ValueError: Can't find 'adapter_config.json' at '/content/drive/MyDrive/Colab_Notebooks/CSCI 5922/final project/llama2_all_data'

#### Old way which gives an error

In [ ]:
def run_inference(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=500, temperature=0.0001, do_sample = False)
    return tokenizer.decode(outputs[0])

In [ ]:

# def evaluate(instructions, input=None):
#     prompts = [generate_prompt(instructions) for instruction in instructions]
#     encodings = tokenizer(prompts, return_tensors="pt", padding=True).to('cuda')
#     # input_ids = inputs["input_ids"].cuda()
#     generation_outputs = model.generate(
#         **encodings,
#         generation_config=generation_config,
#         max_new_tokens=256
#     )

#     returns tokenizer.batch_decode(generation_outputs)

In [ ]:
# test_examples = []
# for question_item in test_dataset:
#     item_id = question_item['id']
#     merged_item = {
#         'id': item_id,
#         'question': question_item['question'],
#         'sql_query': question_item['sql_query'],
#         'training_prompt': generate_test_prompt(question_item['question'], question_item['sql_query'])
#     }
#     test_examples.append(merged_item)

In [ ]:
# example = test_examples[4]
# print(example['training_prompt'])

In [ ]:
# llama_model, tokenizer = create_model_and_tokenizer()
# trained_model_llama = PeftModel.from_pretrained(model, OUTPUT_DIR)

In [ ]:
# question = example['question']
# output = example['sql_query']
# prompt = example['training_prompt']
# print(question)
# print(output)


In [ ]:
# query = run_inference(model, prompt)

In [ ]:
# print(query)

In [ ]:
# for query in test_examples[:5]:
#   response = run_inference(model, query['training_prompt'])
#   print("question: ", query['question'])

#   print("\n\n||||||||||||GENERATED OUTPUT||||||||||||\n\n")
#   print(f"{response}")
#   print("\nexpected output: ", query['sql_query'])
#   print("\n |||||||||| \n")

In [ ]:
# import torch.nn as nn
# import transformers

# class CustomTrainer(transformers.Trainer):
#   def compute_loss(self, model, inputs, return_outputs=False):
#     #Define the number of tokens you want to display
#     num_tokens = 25 #this displays info on the actual and predicted tyokens at the end of each....

#     labels = inputs.pop('labels')
#     loss_mask = inputs.pop("loss_mask")

#     # Forward pass
#     outputs = model(**inputs)

#     logits = output.logits

#     # Check for Nan in logits and labels
#     if torch.isnan(logits).any():
#       print("NaN detected in logits")
#       print(logits)

#     # Convert logits to probabilities using softmax function
#     probs = nn.functional.softmax(logits, dim=-1)

#     # Get the most probable tokens
#     predicted_token_ids = torch.argmax(probs, dim=-1)

#     # Compute the loss
#     loss_fct = nn.CrossEntropyLoss(reduction='none')
#     losses = loss_fct(logits.view(-1, self.model.config.vocab_size), labels.view(-1))

#     #Reshaping the losses to have dimensions [ batch_size, seq_length]
#     losses = losses.view(-1, inputs['input_ids'].size(1))

#     #Apply the loss mask
#     masked_loss = losses * loss_mask

#     #Check for NaN in losses and zero in loss_mask.sum()
#     if torch.isnan(losses).any():
#       print("NaN detected in losses")
#       # print(losses)

#     if loss_mask.sum() == 0:
#       print("Sum of loss_mask is zero")
#       return (torch.tensor(0.0).to(loss_mask.device), outputs) if return_outputs else torch.tensor(0.0).to(loss_mask.device) # last section was autogenerated

#     # Aggregate the masked losses
#     loss = masked_loss.sum() / ( loss_mask.sum() + 1e-9) #normalizing by number of tokens

#     #Print formatted tokens
#     batch_size, seq_length = inputs['input_ids'].size()
#     print("-" * 120)
#     print(f"Token analysis for last {num_tokens} tokens:")

#     # header_format = "{:<10}{:<20}{:<20}{:<20}{:<20}{:<30}{:<30}".format("Index", "Input Token",  ) #need

#     for batch_idx in range(batch_size):
#       input_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][batch_idx])
#       predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_ids[batch_idx])
#       true_tokens = tokenizer.convert_ids_to_tokens(labels[batch_idx])

#       for i in range(-num_tokens, 0, 1):
#         index = seq_length + i #correct index based on sequence length

#     return (loss, outputs ) if return_outputs else loss


#   def get_train_dataloader(self):
#     train_dataset = self.train_dataset
#     data_collator = self.data_collator

#     dataloader_params = {
#         "batch_size" : self.args.train_batch_size,
#         "collate_fn" : data_collator,
#         "num_workers" : self.args.dataloader_num_workers,
#         "pin_memory" : self.args.dataloader_pin_memory,
#     }

#     if not isinstance(train_dataset, torch.utils.data.IterableDataset):
#       dataloader_params['sampler'] = self._get_train_sampler()
#       dataloader_params['drop_last'] = self.args.dataloader_drop_last

#     return DataLoader(train_dataset, **dataloader_params)